In this homework, I will use the California Housing Prices data.
I'll work with the _'median_house_value'_ variable, and transform it to a classification task.

In [110]:
#import libraries
import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score, mean_squared_error, mutual_info_score
from sklearn.model_selection import train_test_split

#set a random seed for reproducibility
np.random.seed(42)

%matplotlib inline

In [111]:
#laod the dataset
df_housing = pd.read_csv('housing.csv')
df_housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [112]:
#check for missing values
df_housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

__Data Preparation__

>* Fill in the missing values with 0.
>* Create a new column *rooms_per_household* by dividing the *column total_rooms* by the column *households* from dataframe.
>* Create a new column *bedrooms_per_room* by dividing the column *total_bedrooms* by the column *total_rooms* from dataframe.
>* Create a new column *population_per_household* by dividing the column *population* by the column *households* from dataframe.

In [113]:
df_housing = df_housing.fillna(0)
df_housing['rooms_per_household'] = df_housing['total_rooms'] / df_housing['households']
df_housing['bedrooms_per_room'] = df_housing['total_bedrooms'] / df_housing['total_rooms']
df_housing['population_per_household'] = df_housing['population'] / df_housing['households']
df_housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467


__Question 1:__
What is the most frequent observation (mode) for the column ocean_proximity?

In [114]:
#check for the most frequent observation on ocean_proximity column
df_housing.ocean_proximity.mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

__Split the data__
>* Split the data in train/val/test sets, with 60%/20%/20% distribution.
>* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
>* Make sure that the target value (median_house_value) is not in your dataframe.

In [115]:
#split the data into 80/20 train_test_set/validation set
train_test_set, val_set = train_test_split(df_housing, test_size = 0.2)

In [116]:
#perform another split, 60/20 train_set/test_set
train_set, test_set = train_test_split(train_test_set, test_size = 0.25)

In [117]:
#check the size of the dataset
len(train_set), len(test_set), len(val_set)

(12384, 4128, 4128)

In [118]:
#reset the index
train_set = train_set.reset_index(drop = True)
val_set = val_set.reset_index(drop = True)
test_set = test_set.reset_index(drop = True)

In [119]:
#select the target variable for the three datasets
y_train = train_set.median_house_value
y_test = test_set.median_house_value
y_val = val_set.median_house_value

In [120]:
#drop the target variable from the datasets
X_train = train_set.drop('median_house_value', axis = 1)
X_test = test_set.drop('median_house_value', axis = 1)
X_val = val_set.drop('median_house_value', axis = 1)

__Question 2:__
>* Create the correlation matrix for the numerical features of your train dataset.
>* What are the two features that have the biggest correlation in this dataset?

In [121]:
#create a correlation matrix for the numerical features of the train dataset
corr_matrix = X_train.corr()
corr_matrix

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
longitude,1.000000,-0.925565,-0.092360,0.037764,0.062496,0.092848,0.048963,-0.015035,-0.029271,0.096569,-0.003715
latitude,-0.925565,1.000000,-0.001490,-0.031198,-0.063200,-0.103765,-0.067358,-0.076069,0.113063,-0.120240,0.009735
housing_median_age,-0.092360,-0.001490,1.000000,-0.361997,-0.320940,-0.289843,-0.303136,-0.118928,-0.154011,0.132410,0.018602
total_rooms,0.037764,-0.031198,-0.361997,1.000000,0.928460,0.854559,0.920656,0.195391,0.120559,-0.184905,-0.026355
total_bedrooms,0.062496,-0.063200,-0.320940,0.928460,1.000000,0.873352,0.980713,-0.013463,-0.011416,0.091632,-0.029680
population,0.092848,-0.103765,-0.289843,0.854559,0.873352,1.000000,0.902034,0.002885,-0.076279,0.037859,0.078892
households,0.048963,-0.067358,-0.303136,0.920656,0.980713,0.902034,1.000000,0.009821,-0.087578,0.070056,-0.028938
median_income,-0.015035,-0.076069,-0.118928,0.195391,-0.013463,0.002885,0.009821,1.000000,0.314908,-0.613780,0.027256
rooms_per_household,-0.029271,0.113063,-0.154011,0.120559,-0.011416,-0.076279,-0.087578,0.314908,1.000000,-0.415093,-0.004254
bedrooms_per_room,0.096569,-0.120240,0.132410,-0.184905,0.091632,0.037859,0.070056,-0.613780,-0.415093,1.000000,0.003951


In [122]:
corr_matrix.unstack().sort_values(ascending = False).head(15)

longitude                 longitude                   1.000000
latitude                  latitude                    1.000000
bedrooms_per_room         bedrooms_per_room           1.000000
rooms_per_household       rooms_per_household         1.000000
median_income             median_income               1.000000
households                households                  1.000000
total_bedrooms            total_bedrooms              1.000000
total_rooms               total_rooms                 1.000000
housing_median_age        housing_median_age          1.000000
population                population                  1.000000
population_per_household  population_per_household    1.000000
total_bedrooms            households                  0.980713
households                total_bedrooms              0.980713
total_bedrooms            total_rooms                 0.928460
total_rooms               total_bedrooms              0.928460
dtype: float64

__Make median_house_value binary__
>* We need to turn the median_house_value variable from numeric into binary.
>* Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.

In [123]:
#calculate the mean
average = df_housing.median_house_value.mean()
x = lambda x: 1 if x > average else 0
y_train = y_train.apply(x).values
y_val = y_val.apply(x).values
y_test = y_test.apply(x).values
y_val

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

__Question 3:__
>* Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
>* What is the value of mutual information?
>* Round it to 2 decimal digits using round(score, 2)

In [124]:
#calculate the mutual information score
mi_score = mutual_info_score(train_set.ocean_proximity, y_train)
round(mi_score, 2)

0.1

__Question 4:__
>* Now let's train a logistic regression
>* Remember that we have one categorical variable ocean_proximity in the data. Include it using one-hot encoding.
>* Fit the model on the training dataset.
>* To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
>* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [125]:
train_dict = X_train.to_dict(orient = 'records')
dv = DictVectorizer(sparse = False)
X_train_ = dv.fit_transform(train_dict)
X_train_

array([[1.64693773e-01, 3.05000000e+02, 3.40000000e+01, ...,
        6.37049180e+00, 3.20000000e+02, 1.94300000e+03],
       [2.07920792e-01, 3.90000000e+01, 4.40000000e+01, ...,
        5.17948718e+00, 4.20000000e+01, 2.02000000e+02],
       [2.12471874e-01, 6.08000000e+02, 2.90000000e+01, ...,
        5.11677632e+00, 6.61000000e+02, 3.11100000e+03],
       ...,
       [1.64333852e-01, 3.16000000e+02, 1.50000000e+01, ...,
        6.10443038e+00, 3.17000000e+02, 1.92900000e+03],
       [1.49213075e-01, 4.88000000e+02, 2.50000000e+01, ...,
        6.77049180e+00, 4.93000000e+02, 3.30400000e+03],
       [2.36879433e-01, 1.49000000e+02, 4.60000000e+01, ...,
        4.73154362e+00, 1.67000000e+02, 7.05000000e+02]])

In [126]:
#vectorize and fit the validation set
val_dict = X_val.to_dict(orient = 'records')
X_validate = dv.transform(val_dict)
X_validate

array([[0.00000000e+00, 3.59000000e+02, 2.50000000e+01, ...,
        4.19220056e+00, 0.00000000e+00, 1.50500000e+03],
       [0.00000000e+00, 5.84000000e+02, 3.00000000e+01, ...,
        5.03938356e+00, 0.00000000e+00, 2.94300000e+03],
       [0.00000000e+00, 9.63000000e+02, 5.20000000e+01, ...,
        3.97715472e+00, 0.00000000e+00, 3.83000000e+03],
       ...,
       [1.30868402e-01, 5.68000000e+02, 2.50000000e+01, ...,
        7.23767606e+00, 5.38000000e+02, 4.11100000e+03],
       [1.85879537e-01, 4.74000000e+02, 3.60000000e+01, ...,
        5.28902954e+00, 4.66000000e+02, 2.50700000e+03],
       [2.59093453e-01, 4.48000000e+02, 1.70000000e+01, ...,
        3.98883929e+00, 4.63000000e+02, 1.78700000e+03]])

In [127]:
#train the LogisticRegression model
model = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000)
model.fit(X_train_, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [128]:
#test the model on the validation set
y_pred = model.predict(X_validate)
#calculate the accuracy
accuracy = round(accuracy_score(y_val, y_pred), 2)
accuracy

0.84

__Question 5:__
>* Let's find the least useful feature using the feature elimination technique.
>* Train a model with all these features (using the same parameters as in Q4).
>* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
>* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
>* Which of the feature has the smallest difference?

In [129]:
features = list(X_train.columns)
features

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'ocean_proximity',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [130]:
for feature in features:
    labels = features.copy()
    labels.remove(feature)

    train_dict = X_train[labels].to_dict(orient = 'records')
    dv = DictVectorizer(sparse = False)
    X_train_ = dv.fit_transform(train_dict)

    val_dict = X_val[labels].to_dict(orient = 'records')
    X_validate = dv.transform(val_dict)

    model = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000)
    model.fit(X_train_, y_train)

    y_pred = model.predict(X_validate)

    score = accuracy_score(y_val, y_pred)
    print(f'{feature}, Accuracy: {(score):.4f}, Diff: {(accuracy - score):.4f}')

longitude, Accuracy: 0.8275, Diff: 0.0125
latitude, Accuracy: 0.8307, Diff: 0.0093
housing_median_age, Accuracy: 0.8253, Diff: 0.0147
total_rooms, Accuracy: 0.8348, Diff: 0.0052
total_bedrooms, Accuracy: 0.8324, Diff: 0.0076
population, Accuracy: 0.8147, Diff: 0.0253
households, Accuracy: 0.8285, Diff: 0.0115
median_income, Accuracy: 0.7730, Diff: 0.0670
ocean_proximity, Accuracy: 0.8164, Diff: 0.0236
rooms_per_household, Accuracy: 0.8348, Diff: 0.0052
bedrooms_per_room, Accuracy: 0.8343, Diff: 0.0057
population_per_household, Accuracy: 0.8345, Diff: 0.0055


__Question 6:__
>* For this question, we'll see how to use a linear regression model from Scikit-Learn
>* We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
>* Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
>* This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
>* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits

In [131]:
train_dict = X_train.to_dict(orient = 'records')
dv = DictVectorizer(sparse = False)
X_train_ = dv.fit_transform(train_dict)
X_train_

array([[1.64693773e-01, 3.05000000e+02, 3.40000000e+01, ...,
        6.37049180e+00, 3.20000000e+02, 1.94300000e+03],
       [2.07920792e-01, 3.90000000e+01, 4.40000000e+01, ...,
        5.17948718e+00, 4.20000000e+01, 2.02000000e+02],
       [2.12471874e-01, 6.08000000e+02, 2.90000000e+01, ...,
        5.11677632e+00, 6.61000000e+02, 3.11100000e+03],
       ...,
       [1.64333852e-01, 3.16000000e+02, 1.50000000e+01, ...,
        6.10443038e+00, 3.17000000e+02, 1.92900000e+03],
       [1.49213075e-01, 4.88000000e+02, 2.50000000e+01, ...,
        6.77049180e+00, 4.93000000e+02, 3.30400000e+03],
       [2.36879433e-01, 1.49000000e+02, 4.60000000e+01, ...,
        4.73154362e+00, 1.67000000e+02, 7.05000000e+02]])

In [132]:
val_dict = X_val.to_dict(orient = 'records')
X_validate = dv.transform(val_dict)
X_validate

array([[0.00000000e+00, 3.59000000e+02, 2.50000000e+01, ...,
        4.19220056e+00, 0.00000000e+00, 1.50500000e+03],
       [0.00000000e+00, 5.84000000e+02, 3.00000000e+01, ...,
        5.03938356e+00, 0.00000000e+00, 2.94300000e+03],
       [0.00000000e+00, 9.63000000e+02, 5.20000000e+01, ...,
        3.97715472e+00, 0.00000000e+00, 3.83000000e+03],
       ...,
       [1.30868402e-01, 5.68000000e+02, 2.50000000e+01, ...,
        7.23767606e+00, 5.38000000e+02, 4.11100000e+03],
       [1.85879537e-01, 4.74000000e+02, 3.60000000e+01, ...,
        5.28902954e+00, 4.66000000e+02, 2.50700000e+03],
       [2.59093453e-01, 4.48000000e+02, 1.70000000e+01, ...,
        3.98883929e+00, 4.63000000e+02, 1.78700000e+03]])

In [133]:
y_train = np.log1p(train_set.median_house_value.values)
y_val = np.log1p(val_set.median_house_value.values)
y_test = np.log1p(test_set.median_house_value.values)

In [134]:
alpha_values = [0, 0.01, 0.1, 1, 10]

for a in alpha_values:
    model = Ridge(alpha = a, solver = "sag")
    model.fit(X_train_, y_train)
    
    y_pred = model.predict(X_validate)
    
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    
    print(f'alpha: {a}, RMSE: {(score):.3f}')


alpha: 0, RMSE: 0.568
alpha: 0.01, RMSE: 0.568
alpha: 0.1, RMSE: 0.568
alpha: 1, RMSE: 0.568
alpha: 10, RMSE: 0.568
